In [1]:
#Mounting Google Drive in Colab notebook
from google.colab import drive
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


**LIBRARY IMPORTS**

In [2]:
from numpy.random import seed

# Ignore specific warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

#Importing Basic Libaries for Data Load and Image Movement
import os
import pandas as pd
import numpy as np
import itertools
import shutil

#Tensorflow libraries
import tensorflow
tensorflow.random.set_seed(101)

#libraries to build neural network models
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,GlobalAveragePooling2D,BatchNormalization
from keras import backend as k

from keras.utils import to_categorical
from keras import optimizers
from keras.optimizers import SGD,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.utils import img_to_array,array_to_img
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
from tensorflow.keras import regularizers

#Model Libraries:
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet  import preprocess_input

#Split and Importing evaluation libraries
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.metrics import confusion_matrix, roc_curve, auc,classification_report,balanced_accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer


#Plotting Graphs:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.densenet import preprocess_input

**DIRECTORIES CREATION**

In [4]:
# Creating new directory for file movement:
skin_dense_base_dir = 'skin_dense_base_dir'
os.mkdir(skin_dense_base_dir)

In [5]:
# Generate a pathway leading to the 'skin_dense_base_dir' to which we will append the names of the new folders
# Training Directory creation
train_dir = os.path.join(skin_dense_base_dir, 'train_dir')
os.mkdir(train_dir)

# Validation Directory creation
val_dir = os.path.join(skin_dense_base_dir, 'val_dir')
os.mkdir(val_dir)

# Test Directory creation
test_dir = os.path.join(skin_dense_base_dir, 'test_dir')
os.mkdir(test_dir)

In [6]:
# Creation of new folders inside Training Directory
nv = os.path.join(train_dir, 'nv')
os.mkdir(nv)
mel = os.path.join(train_dir, 'mel')
os.mkdir(mel)
bkl = os.path.join(train_dir, 'bkl')
os.mkdir(bkl)
bcc = os.path.join(train_dir, 'bcc')
os.mkdir(bcc)
akiec = os.path.join(train_dir, 'akiec')
os.mkdir(akiec)
vasc = os.path.join(train_dir, 'vasc')
os.mkdir(vasc)
df = os.path.join(train_dir, 'df')
os.mkdir(df)

In [7]:
# Creation of new folders inside Validation Directory
nv = os.path.join(val_dir, 'nv')
os.mkdir(nv)
mel = os.path.join(val_dir, 'mel')
os.mkdir(mel)
bkl = os.path.join(val_dir, 'bkl')
os.mkdir(bkl)
bcc = os.path.join(val_dir, 'bcc')
os.mkdir(bcc)
akiec = os.path.join(val_dir, 'akiec')
os.mkdir(akiec)
vasc = os.path.join(val_dir, 'vasc')
os.mkdir(vasc)
df = os.path.join(val_dir, 'df')
os.mkdir(df)

In [8]:
# create new folders inside test_dir
nv = os.path.join(test_dir, 'nv')
os.mkdir(nv)
mel = os.path.join(test_dir, 'mel')
os.mkdir(mel)
bkl = os.path.join(test_dir, 'bkl')
os.mkdir(bkl)
bcc = os.path.join(test_dir, 'bcc')
os.mkdir(bcc)
akiec = os.path.join(test_dir, 'akiec')
os.mkdir(akiec)
vasc = os.path.join(test_dir, 'vasc')
os.mkdir(vasc)
df = os.path.join(test_dir, 'df')
os.mkdir(df)

**DATA LOAD AND ANALYSIS**

In [9]:
#Read metadata information of HAM10000 dataset
df_data = pd.read_csv('/content/drive/MyDrive/DISSERTATION/HAM10000_DATASET/HAM10000_metadata')

df_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [10]:
df_data.shape

(10015, 8)

In [11]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lesion_id     10015 non-null  object 
 1   image_id      10015 non-null  object 
 2   dx            10015 non-null  object 
 3   dx_type       10015 non-null  object 
 4   age           9958 non-null   float64
 5   sex           10015 non-null  object 
 6   localization  10015 non-null  object 
 7   dataset       10015 non-null  object 
dtypes: float64(1), object(7)
memory usage: 626.1+ KB


In [12]:
#HAM1000 attributes basic analysis
print('unique lesion_id records:' ,df_data.lesion_id.nunique())
print('unique image_id records:' ,df_data.image_id.nunique())
print('unique dx records:' ,df_data.dx.nunique())
print('unique dx_type records:' ,df_data.dx_type.nunique())
print('unique  age records:' ,df_data.age.nunique())
print('unique  sex records:' ,df_data.sex.nunique())
print('unique  localization records:' ,df_data.localization.nunique())
print('unique   dataset records:' ,df_data. dataset.nunique())

unique lesion_id records: 7470
unique image_id records: 10015
unique dx records: 7
unique dx_type records: 4
unique  age records: 18
unique  sex records: 3
unique  localization records: 15
unique   dataset records: 4


In [13]:
#Sorting the dataset based on Lesion_Id
df_data = df_data.sort_values(by='lesion_id')

In [14]:
# Train , Test and Validation Split Ratio
train_ratio = 0.7  # 70% of the data for training
test_ratio = 0.15  # 15% of the data for testing
val_ratio = 0.15   # 15% of the data for validation

In [15]:
# Split the DataFrame into train, test, and validation dataframes
df_train, test_val_df = train_test_split(df_data, test_size=(test_ratio + val_ratio), random_state=42)
df_test, df_val = train_test_split(test_val_df, test_size=(val_ratio / (test_ratio + val_ratio)), random_state=42)


In [16]:
#Length of Data Frame
print('Length of df_train:',len(df_train))
print('Length of df_val:',len(df_val))
print('Length of df_test:',len(df_test))

Length of df_train: 7010
Length of df_val: 1503
Length of df_test: 1502


In [17]:
#Training set unique class values
df_train['dx'].value_counts()

nv       4647
bkl       799
mel       790
bcc       362
akiec     228
vasc      100
df         84
Name: dx, dtype: int64

In [18]:
#Validation set unique class values
df_val['dx'].value_counts()

nv       1015
mel       182
bkl       151
bcc        70
akiec      54
vasc       18
df         13
Name: dx, dtype: int64

In [19]:
#Test set unique class values
df_test['dx'].value_counts()

nv       1043
bkl       149
mel       141
bcc        82
akiec      45
vasc       24
df         18
Name: dx, dtype: int64

**IMAGES COPY FROM SOURCE TO DESTINATION**

In [20]:
# Get a list of images from HAM folder in Google Drive
folder = os.listdir('/content/drive/MyDrive/DISSERTATION/HAM10000_DATASET/ALL_TRAIN_IMAGES')

# Get a list of train ,test and val images
train_list = list(df_train['image_id'])
val_list = list(df_val['image_id'])
test_list = list(df_test['image_id'])

In [21]:
#Moving the images from google dir to train_dir
for image in train_list:

    fname = image + '.jpg'
    #label = df_data.loc[image,'dx']
    label=df_data.loc[df_data['image_id'] == image, 'dx'].values[0]


    if fname in folder:
        # source path to image
        src = os.path.join('/content/drive/MyDrive/DISSERTATION/HAM10000_DATASET/ALL_TRAIN_IMAGES', fname)
        # destination path to image
        dst = os.path.join(train_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [22]:
#Moving the images from google dir to val_dir
for image in val_list:

    fname = image + '.jpg'
    #label = df_data.loc[image,'dx']
    label=df_data.loc[df_data['image_id'] == image, 'dx'].values[0]

    if fname in folder:
        # source path to image
        src = os.path.join('/content/drive/MyDrive/DISSERTATION/HAM10000_DATASET/ALL_TRAIN_IMAGES', fname)
        # destination path to image
        dst = os.path.join(val_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [23]:
#Moving the images from google dir to test_dir
for image in test_list:

    fname = image + '.jpg'
    #label = df_data.loc[image,'dx']
    label=df_data.loc[df_data['image_id'] == image, 'dx'].values[0]

    if fname in folder:
        # source path to image
        src = os.path.join('/content/drive/MyDrive/DISSERTATION/HAM10000_DATASET/ALL_TRAIN_IMAGES', fname)
        # destination path to image
        dst = os.path.join(test_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [24]:
# Determine the count of training images present in each directory.

print('nv:',len(os.listdir('skin_dense_base_dir/train_dir/nv')))
print('mel:',len(os.listdir('skin_dense_base_dir/train_dir/mel')))
print('bkl:',len(os.listdir('skin_dense_base_dir/train_dir/bkl')))
print('bcc:',len(os.listdir('skin_dense_base_dir/train_dir/bcc')))
print('akiec:',len(os.listdir('skin_dense_base_dir/train_dir/akiec')))
print('vasc:',len(os.listdir('skin_dense_base_dir/train_dir/vasc')))
print('df:',len(os.listdir('skin_dense_base_dir/train_dir/df')))

nv: 4647
mel: 790
bkl: 799
bcc: 362
akiec: 228
vasc: 100
df: 84


In [25]:
# Determine the count of val images present in each directory.

print('nv:',len(os.listdir('skin_dense_base_dir/val_dir/nv')))
print('mel:',len(os.listdir('skin_dense_base_dir/val_dir/mel')))
print('bkl:',len(os.listdir('skin_dense_base_dir/val_dir/bkl')))
print('bcc:',len(os.listdir('skin_dense_base_dir/val_dir/bcc')))
print('akiec:',len(os.listdir('skin_dense_base_dir/val_dir/akiec')))
print('vasc:',len(os.listdir('skin_dense_base_dir/val_dir/vasc')))
print('df:',len(os.listdir('skin_dense_base_dir/val_dir/df')))

nv: 1015
mel: 182
bkl: 151
bcc: 70
akiec: 54
vasc: 18
df: 13


In [26]:
# Check how many test images we have in each folder.

print('nv:',len(os.listdir('skin_dense_base_dir/test_dir/nv')))
print('mel:',len(os.listdir('skin_dense_base_dir/test_dir/mel')))
print('bkl:',len(os.listdir('skin_dense_base_dir/test_dir/bkl')))
print('bcc:',len(os.listdir('skin_dense_base_dir/test_dir/bcc')))
print('akiec:',len(os.listdir('skin_dense_base_dir/test_dir/akiec')))
print('vasc:',len(os.listdir('skin_dense_base_dir/test_dir/vasc')))
print('df:',len(os.listdir('skin_dense_base_dir/test_dir/df')))

nv: 1043
mel: 141
bkl: 149
bcc: 82
akiec: 45
vasc: 24
df: 18


In [27]:
densenet_model = tf.keras.models.load_model('/content/drive/MyDrive/DISSERTATION/HAM10000_DATASET/MODEL_OUTPUTS/SKIN_CANCER_DENSENET_201_Final_Model.h5')

In [28]:
# Get the list of class names (subfolders)
class_names = sorted(os.listdir(test_dir))
class_names

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [29]:
# Get the list of class names (subfolders)
class_names = sorted(os.listdir(test_dir))
class_names

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [34]:
# Sample Image prediction
for class_name in class_names:
    class_path = os.path.join(test_dir, class_name)
    image_files = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

In [35]:
image_files

['ISIC_0025596.jpg',
 'ISIC_0025599.jpg',
 'ISIC_0033158.jpg',
 'ISIC_0033123.jpg',
 'ISIC_0026876.jpg',
 'ISIC_0030283.jpg',
 'ISIC_0030722.jpg',
 'ISIC_0031648.jpg',
 'ISIC_0032932.jpg',
 'ISIC_0024402.jpg',
 'ISIC_0032076.jpg',
 'ISIC_0031217.jpg',
 'ISIC_0032409.jpg',
 'ISIC_0027159.jpg',
 'ISIC_0031950.jpg',
 'ISIC_0026393.jpg',
 'ISIC_0029099.jpg',
 'ISIC_0027385.jpg',
 'ISIC_0033092.jpg',
 'ISIC_0033817.jpg',
 'ISIC_0028680.jpg',
 'ISIC_0033749.jpg',
 'ISIC_0034214.jpg',
 'ISIC_0025578.jpg']

In [36]:
from keras.preprocessing import image

In [37]:
for image_file in image_files:
    image_path = os.path.join(class_path, image_file)

    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    image_files_without_extension = image_file.split('.')
    predictions = densenet_model.predict(img_array)
    class_index = np.argmax(predictions, axis=1)[0]

    # Print the predicted class index and class name
    print(f"Image: {image_file}, Predicted Class: {class_names[class_index]}")
    print('Actual Class ',df_data[df_data['image_id'] == image_files_without_extension[0]]['dx'])

1/1 [==============================] - 8s 8s/step
Image: ISIC_0025596.jpg, Predicted Class: vasc
Actual Class  2420    vasc
Name: dx, dtype: object
1/1 [==============================] - 0s 38ms/step
Image: ISIC_0025599.jpg, Predicted Class: vasc
Actual Class  2333    vasc
Name: dx, dtype: object
1/1 [==============================] - 0s 38ms/step
Image: ISIC_0033158.jpg, Predicted Class: vasc
Actual Class  2357    vasc
Name: dx, dtype: object
1/1 [==============================] - 0s 36ms/step
Image: ISIC_0033123.jpg, Predicted Class: vasc
Actual Class  2375    vasc
Name: dx, dtype: object
1/1 [==============================] - 0s 36ms/step
Image: ISIC_0026876.jpg, Predicted Class: vasc
Actual Class  2388    vasc
Name: dx, dtype: object
1/1 [==============================] - 0s 38ms/step
Image: ISIC_0030283.jpg, Predicted Class: vasc
Actual Class  2411    vasc
Name: dx, dtype: object
1/1 [==============================] - 0s 36ms/step
Image: ISIC_0030722.jpg, Predicted Class: vasc
Act